# Feature Engineering & Model's

## Libraries

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import re

/Users/ulisesgordillo/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Import data

In [2]:
#get cwd    
import os
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

#change to  /Users/ulisesgordillo/Downloads/capstone/data
os.chdir('/Users/ulisesgordillo/Downloads/capstone/clean_data')
cwd = os.getcwd()
print(f"Current working directory: {cwd}")



Current working directory: /Users/ulisesgordillo/Downloads/capstone/notebooks
Current working directory: /Users/ulisesgordillo/Downloads/capstone/clean_data


In [3]:
#read data with polars
df = pl.read_excel('full_join.xlsx', sheet_name='Sheet1')

Could not determine dtype for column 13, falling back to string
Could not determine dtype for column 14, falling back to string
Could not determine dtype for column 15, falling back to string


In [4]:
df.shape


(475629, 23)

In [5]:
df.head(5)

ReportID,ProductReference,DateOfReport,SupplierName,GarmentType,Material,Weight,ProposedUnitsPerCarton,ProposedFoldingMethod,ProposedLayout,PackagingQuality,Size,Collection,IssueDescription,ResolutionStatus,CostImpact (€),PreviousMonth,PackagesHandled,TotalIncidents,AnomaliesDetected,BadPackagingRate (%),OnTimeDeliveryRate (%),AverageCostPerIncident (€)
str,str,date,str,str,str,f64,i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,f64,f64,f64
"""RPT0000001""","""PRD07271""",2024-03-04,"""SupplierA""","""Pants""","""Polyester""",0.35,29,"""Method2""","""LayoutC""","""Good""","""S""","""Winter""",null,null,null,"""2024-02""",7541,173,20,8.09,86.03,509.15
"""RPT0000002""","""PRD00861""",2024-05-27,"""SupplierC""","""T-Shirt""","""Denim""",0.21,20,"""Method2""","""LayoutB""","""Good""","""M""","""Winter""",null,null,null,"""2024-04""",4216,181,32,27.26,75.51,527.23
"""RPT0000003""","""PRD05391""",2023-11-18,"""SupplierA""","""Shirt""","""Cotton""",0.2,31,"""Method1""","""LayoutA""","""Good""","""S""","""Winter""",null,null,null,"""2023-10""",7933,167,22,8.7,85.31,534.7
"""RPT0000004""","""PRD05192""",2024-06-13,"""SupplierA""","""Coat""","""Cotton""",1.3,5,"""Method1""","""LayoutD""","""Good""","""XL""","""Winter""",null,null,null,"""2024-05""",8002,158,19,8.53,88.26,531.85
"""RPT0000005""","""PRD05735""",2023-07-18,"""SupplierA""","""Coat""","""Polyester""",1.11,9,"""Method2""","""LayoutD""","""Good""","""M""","""Autumn""",null,null,null,"""2023-06""",7669,154,22,8.4,91.79,629.89


In [6]:
#how many nulls we have in each column  
df_nulls = df.select(pl.all().is_null().sum())
df_nulls = df_nulls.to_pandas()
df_nulls = df_nulls.T
df_nulls

,0
ReportID,0
ProductReference,0
DateOfReport,0
SupplierName,0
GarmentType,0
Material,0
Weight,0
ProposedUnitsPerCarton,0
ProposedFoldingMethod,0
ProposedLayout,0


## Data Manipulation

### Target Variable

In [7]:
y = df['PackagingQuality'].map_elements(lambda x: 1 if x == 'Good' else 0, return_dtype=pl.Int8)
x= df.drop('PackagingQuality')

In [8]:
#print x and y shape
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: (475629, 22)
y shape: (475629,)


### Removing columns

In [9]:
#remove possible unimportant columns
columns_to_drop = ['ReportID','ProductReference','IssueDescription','ResolutionStatus','CostImpact (€)']
x = x.drop(columns_to_drop)

In [10]:
x

DateOfReport,SupplierName,GarmentType,Material,Weight,ProposedUnitsPerCarton,ProposedFoldingMethod,ProposedLayout,Size,Collection,PreviousMonth,PackagesHandled,TotalIncidents,AnomaliesDetected,BadPackagingRate (%),OnTimeDeliveryRate (%),AverageCostPerIncident (€)
date,str,str,str,f64,i64,str,str,str,str,str,i64,i64,i64,f64,f64,f64
2024-03-04,"""SupplierA""","""Pants""","""Polyester""",0.35,29,"""Method2""","""LayoutC""","""S""","""Winter""","""2024-02""",7541,173,20,8.09,86.03,509.15
2024-05-27,"""SupplierC""","""T-Shirt""","""Denim""",0.21,20,"""Method2""","""LayoutB""","""M""","""Winter""","""2024-04""",4216,181,32,27.26,75.51,527.23
2023-11-18,"""SupplierA""","""Shirt""","""Cotton""",0.2,31,"""Method1""","""LayoutA""","""S""","""Winter""","""2023-10""",7933,167,22,8.7,85.31,534.7
2024-06-13,"""SupplierA""","""Coat""","""Cotton""",1.3,5,"""Method1""","""LayoutD""","""XL""","""Winter""","""2024-05""",8002,158,19,8.53,88.26,531.85
2023-07-18,"""SupplierA""","""Coat""","""Polyester""",1.11,9,"""Method2""","""LayoutD""","""M""","""Autumn""","""2023-06""",7669,154,22,8.4,91.79,629.89
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-11-27,"""SupplierB""","""Skirt""","""Cotton""",0.3,24,"""Method2""","""LayoutB""","""M""","""Spring""","""2023-10""",6265,195,22,15.13,82.26,541.18
2023-10-14,"""SupplierD""","""T-Shirt""","""Polyester""",0.12,35,"""Method2""","""LayoutA""","""L""","""Summer""","""2023-09""",2953,153,31,32.71,68.93,520.87
2024-04-03,"""SupplierA""","""Jacket""","""Cotton""",0.91,17,"""Method2""","""LayoutD""","""S""","""Winter""","""2024-03""",8039,180,30,7.95,87.94,601.26


### Label Encoding

In [11]:

date_col = 'DateOfReport' # The actual name of your date column in X
categorical_cols = [
    'SupplierName',
    'GarmentType',
    'Material',
    'ProposedFoldingMethod',
    'ProposedLayout',
    'Size',
    'Collection',
]

categorical_cols = [col for col in categorical_cols if col in x.columns]

# --- Step 1: Date Feature Extraction ---

print(f"Extracting features from date column: '{date_col}'")
if date_col in x.columns:
    x = x.with_columns([
        pl.col(date_col).dt.year().alias('report_year'),
        pl.col(date_col).dt.month().alias('report_month'),
        pl.col(date_col).dt.day().alias('report_day'),
        pl.col(date_col).dt.weekday().alias('report_weekday'), # Monday=1, Sunday=7 in Polars (adjust trig if needed or use weekday for Mon=0)
        pl.col(date_col).dt.ordinal_day().alias('report_day_of_year'),
        pl.col(date_col).dt.week().alias('report_week_of_year'),
        # Add other extractions if needed (e.g., .hour(), .minute() if it was datetime)
    ])

    # Optional: Encode cyclical features (Month, DayOfWeek) using sin/cos
    # Adjust division factor if weekday() starts at 1 (divide by 7 still okay)
    print("Encoding cyclical date features (month, weekday)...")
    x = x.with_columns([
        (np.sin(2 * np.pi * pl.col('report_month') / 12)).alias('report_month_sin'),
        (np.cos(2 * np.pi * pl.col('report_month') / 12)).alias('report_month_cos'),
        (np.sin(2 * np.pi * pl.col('report_weekday') / 7)).alias('report_weekday_sin'),
        (np.cos(2 * np.pi * pl.col('report_weekday') / 7)).alias('report_weekday_cos'),
    ])

    # Optional: Drop original date and intermediate columns if desired
    print(f"Dropping original date column '{date_col}' and intermediate month/weekday.")
    x = x.drop([date_col, 'report_month', 'report_weekday'])

else:
    print(f"Warning: Date column '{date_col}' not found in X. Skipping date feature extraction.")

# --- Step 2: Categorical Feature Encoding ---

print(f"Encoding categorical columns: {categorical_cols}")
if categorical_cols:
    # Convert specified string columns to Polars Categorical type
    # This is often efficient for tree-based models like LightGBM/XGBoost
    for col_name in categorical_cols:
        x = x.with_columns(
            pl.col(col_name).cast(pl.Categorical).alias(col_name)
        )
    print("Categorical columns cast to pl.Categorical.")
else:
    print("No categorical columns specified for encoding.")





Extracting features from date column: 'DateOfReport'
Encoding cyclical date features (month, weekday)...
Dropping original date column 'DateOfReport' and intermediate month/weekday.
Encoding categorical columns: ['SupplierName', 'GarmentType', 'Material', 'ProposedFoldingMethod', 'ProposedLayout', 'Size', 'Collection']
Categorical columns cast to pl.Categorical.


In [12]:
x

SupplierName,GarmentType,Material,Weight,ProposedUnitsPerCarton,ProposedFoldingMethod,ProposedLayout,Size,Collection,PreviousMonth,PackagesHandled,TotalIncidents,AnomaliesDetected,BadPackagingRate (%),OnTimeDeliveryRate (%),AverageCostPerIncident (€),report_year,report_day,report_day_of_year,report_week_of_year,report_month_sin,report_month_cos,report_weekday_sin,report_weekday_cos
cat,cat,cat,f64,i64,cat,cat,cat,cat,str,i64,i64,i64,f64,f64,f64,i32,i8,i16,i8,f64,f64,f64,f64
"""SupplierA""","""Pants""","""Polyester""",0.35,29,"""Method2""","""LayoutC""","""S""","""Winter""","""2024-02""",7541,173,20,8.09,86.03,509.15,2024,4,64,10,1.0,6.1232e-17,0.781831,0.62349
"""SupplierC""","""T-Shirt""","""Denim""",0.21,20,"""Method2""","""LayoutB""","""M""","""Winter""","""2024-04""",4216,181,32,27.26,75.51,527.23,2024,27,148,22,0.5,-0.866025,0.781831,0.62349
"""SupplierA""","""Shirt""","""Cotton""",0.2,31,"""Method1""","""LayoutA""","""S""","""Winter""","""2023-10""",7933,167,22,8.7,85.31,534.7,2023,18,322,46,-0.5,0.866025,-0.781831,0.62349
"""SupplierA""","""Coat""","""Cotton""",1.3,5,"""Method1""","""LayoutD""","""XL""","""Winter""","""2024-05""",8002,158,19,8.53,88.26,531.85,2024,13,165,24,1.2246e-16,-1.0,-0.433884,-0.900969
"""SupplierA""","""Coat""","""Polyester""",1.11,9,"""Method2""","""LayoutD""","""M""","""Autumn""","""2023-06""",7669,154,22,8.4,91.79,629.89,2023,18,199,29,-0.5,-0.866025,0.974928,-0.222521
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SupplierB""","""Skirt""","""Cotton""",0.3,24,"""Method2""","""LayoutB""","""M""","""Spring""","""2023-10""",6265,195,22,15.13,82.26,541.18,2023,27,331,48,-0.5,0.866025,0.781831,0.62349
"""SupplierD""","""T-Shirt""","""Polyester""",0.12,35,"""Method2""","""LayoutA""","""L""","""Summer""","""2023-09""",2953,153,31,32.71,68.93,520.87,2023,14,287,41,-0.866025,0.5,-0.781831,0.62349
"""SupplierA""","""Jacket""","""Cotton""",0.91,17,"""Method2""","""LayoutD""","""S""","""Winter""","""2024-03""",8039,180,30,7.95,87.94,601.26,2024,3,94,14,0.866025,-0.5,0.433884,-0.900969
